### 주요 하이퍼파라미터

1. **penalty**: 사용될 규제(regularization) 유형을 지정합니다. 기본값은 'l2'이며, 'l1', 'l2', 'elasticnet', 'none' 중 하나를 선택할 수 있습니다.
   - `'l1'`: L1 규제(Lasso)
   - `'l2'`: L2 규제(Ridge)
   - `'elasticnet'`: L1과 L2의 결합
   - `'none'`: 규제 없음

2. **dual**: 듀얼 형식을 사용할지 여부를 지정합니다. 기본값은 `False`입니다. 주로 샘플 수가 특성 수보다 많은 경우에 `False`로 설정합니다.

3. **tol**: 종료 기준을 설정하는 데 사용되는 허용 오차입니다. 기본값은 `1e-4`입니다.

4. **C**: 규제 강도를 설정합니다. 값이 작을수록 강한 규제를 의미합니다. 기본값은 `1.0`입니다.

5. **fit_intercept**: 절편을 추가할지 여부를 지정합니다. 기본값은 `True`입니다.

6. **intercept_scaling**: `fit_intercept=True`일 때 절편 항에 대한 스케일링 값입니다. 기본값은 `1`입니다.

7. **class_weight**: 클래스 가중치를 지정합니다. 기본값은 `None`이며, `balanced`로 설정할 수 있습니다.

8. **random_state**: 난수 생성기를 설정합니다. 결과의 재현성을 위해 사용됩니다.

9. **solver**: 최적화 알고리즘을 지정합니다. 기본값은 `'lbfgs'`입니다. 가능한 값으로는 `'newton-cg'`, `'lbfgs'`, `'liblinear'`, `'sag'`, `'saga'`가 있습니다.
   - `'newton-cg'`: Newton의 방법 변형
   - `'lbfgs'`: Broyden-Fletcher-Goldfarb-Shanno (BFGS) 알고리즘 변형
   - `'liblinear'`: 작은 데이터셋에 적합
   - `'sag'`: 대규모 데이터셋에 적합
   - `'saga'`: 매우 대규모 데이터셋에 적합

10. **max_iter**: 최대 반복 횟수입니다. 기본값은 `100`입니다.

11. **multi_class**: 다중 클래스 설정 방법을 지정합니다. 기본값은 `'auto'`입니다.
    - `'auto'`: 이진 분류에서는 이진 로지스틱 회귀를 사용하고, 다중 분류에서는 OvR을 사용합니다.
    - `'ovr'`: 일대다(One-vs-Rest) 전략
    - `'multinomial'`: 다항 로지스틱 회귀

12. **verbose**: 출력할 로깅 정보의 양을 설정합니다. 기본값은 `0`입니다.

13. **warm_start**: 이전 학습의 결과를 초기화에 사용할지 여부를 지정합니다. 기본값은 `False`입니다.

14. **n_jobs**: 병렬 작업에 사용할 CPU 코어 수를 지정합니다. 기본값은 `None`입니다.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report


- 타이타닉 데이터셋으로 1차로 성능 평가
- 위에 있는 하이퍼파라미터를 조금 조정해서 성능을 올려보는 것

In [ ]:
#타이타닉 불러오기
df=sns.load_dataset('titanic')

In [ ]:
#결측치 정리

df=df.dropna(subset=['age','embarked'])

#데이터 전처리
df['sex'] = df['sex'].map({'male':0, 'female':1})
df =pd.get_dummies(df, columns = ['embarked'],drop_first=True)


In [ ]:
# 특성 x, y 분리
X= df[['sex','age','fare','embarked_Q','embarked_S']]
y = df['survived']

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=111)


In [ ]:
#특성 스케일링
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [ ]:
#로지스틱 회귀 모델 실습
#Base Model
model=LogisticRegression()
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)
y_pred_prob=model.predict_proba(X_test)[:,1]

In [ ]:
##성능평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc =roc_auc_score(y_test, y_pred)

In [ ]:
## 혼동행렬, 분류 보고서 출력
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [ ]:
print(conf_matrix)
print(class_report)
print(accuracy_score(y_test, y_pred))

[[67 11]
 [20 45]]
              precision    recall  f1-score   support

           0       0.77      0.86      0.81        78
           1       0.80      0.69      0.74        65

    accuracy                           0.78       143
   macro avg       0.79      0.78      0.78       143
weighted avg       0.79      0.78      0.78       143

0.7832167832167832


### 하이퍼파라미터 튜닝을 통해 성능 조금 개선해 보자!

In [ ]:
#로지스틱 회귀 모델 실습
#Base Model
model_t1=LogisticRegression(
            penalty='l2',
            solver='liblinear',
            class_weight = 'balanced',
            max_iter = 100)
model_t1.fit(X_train, y_train)

# 예측
y_pred = model_t1.predict(X_test)
y_pred_prob=model_t1.predict_proba(X_test)[:,1]

In [ ]:
##하이퍼파라미터 튜닝 한 성능평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc =roc_auc_score(y_test, y_pred)

In [ ]:
## 혼동행렬, 분류 보고서 출력
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [ ]:
print(conf_matrix)
print(class_report)
print(accuracy_score(y_test, y_pred))

[[67 11]
 [19 46]]
              precision    recall  f1-score   support

           0       0.78      0.86      0.82        78
           1       0.81      0.71      0.75        65

    accuracy                           0.79       143
   macro avg       0.79      0.78      0.79       143
weighted avg       0.79      0.79      0.79       143

0.7902097902097902


In [ ]:
df

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,deck,embark_town,alive,alone,embarked_Q,embarked_S
0,0,3,0,22.0,1,0,7.2500,Third,man,True,NaN,Southampton,no,False,0,1
1,1,1,1,38.0,1,0,71.2833,First,woman,False,C,Cherbourg,yes,False,0,0
2,1,3,1,26.0,0,0,7.9250,Third,woman,False,NaN,Southampton,yes,True,0,1
3,1,1,1,35.0,1,0,53.1000,First,woman,False,C,Southampton,yes,False,0,1
4,0,3,0,35.0,0,0,8.0500,Third,man,True,NaN,Southampton,no,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,1,39.0,0,5,29.1250,Third,woman,False,NaN,Queenstown,no,False,1,0
886,0,2,0,27.0,0,0,13.0000,Second,man,True,NaN,Southampton,no,True,0,1
887,1,1,1,19.0,0,0,30.0000,First,woman,False,B,Southampton,yes,True,0,1
889,1,1,0,26.0,0,0,30.0000,First,man,True,C,Cherbourg,yes,True,0,0


## BMI 예측을 해보자!

In [ ]:
#BMI 데이터로 분석!
df=pd.read_csv('heart_2020_cleaned.csv')

In [ ]:
# 사용할 컬럼 정리
df2 =pd.get_dummies(df, columns = ['HeartDisease','Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','AgeCategory','Race','Diabetic','PhysicalActivity','GenHealth','Asthma','KidneyDisease','SkinCancer'], drop_first=True)

In [ ]:
df2.columns

Index(['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime',
       'HeartDisease_Yes', 'Smoking_Yes', 'AlcoholDrinking_Yes', 'Stroke_Yes',
       'DiffWalking_Yes', 'Sex_Male', 'AgeCategory_25-29', 'AgeCategory_30-34',
       'AgeCategory_35-39', 'AgeCategory_40-44', 'AgeCategory_45-49',
       'AgeCategory_50-54', 'AgeCategory_55-59', 'AgeCategory_60-64',
       'AgeCategory_65-69', 'AgeCategory_70-74', 'AgeCategory_75-79',
       'AgeCategory_80 or older', 'Race_Asian', 'Race_Black', 'Race_Hispanic',
       'Race_Other', 'Race_White', 'Diabetic_No, borderline diabetes',
       'Diabetic_Yes', 'Diabetic_Yes (during pregnancy)',
       'PhysicalActivity_Yes', 'GenHealth_Fair', 'GenHealth_Good',
       'GenHealth_Poor', 'GenHealth_Very good', 'Asthma_Yes',
       'KidneyDisease_Yes', 'SkinCancer_Yes'],
      dtype='object')

In [ ]:
##스케일링 작업
df_num = df2[['BMI','PhysicalHealth','MentalHealth','SleepTime']]
df_nom = df2.drop(['BMI','PhysicalHealth','MentalHealth','SleepTime'],axis=1)

In [ ]:
## Standard 스케일링
scaler=StandardScaler()
df_scaler=scaler.fit_transform(df_num)

In [ ]:
df_num2 =pd.DataFrame(data= df_scaler, columns= df_num.columns)

In [ ]:
df_new=pd.concat([df_num2, df_nom], axis=1)

In [ ]:
df_new

,BMI,PhysicalHealth,MentalHealth,SleepTime,HeartDisease_Yes,Smoking_Yes,AlcoholDrinking_Yes,Stroke_Yes,DiffWalking_Yes,Sex_Male,...,Diabetic_Yes,Diabetic_Yes (during pregnancy),PhysicalActivity_Yes,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_Yes,KidneyDisease_Yes,SkinCancer_Yes
0,-1.844750,-0.046751,3.281069,-1.460354,0,1,0,0,0,0,...,1,0,1,0,0,0,1,1,0,1
1,-1.256338,-0.424070,-0.490039,-0.067601,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,-0.274603,2.091388,3.281069,0.628776,0,1,0,0,0,1,...,1,0,1,1,0,0,0,1,0,0
3,-0.647473,-0.424070,-0.490039,-0.763977,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,-0.726138,3.097572,-0.490039,0.628776,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,-0.144019,0.456341,-0.490039,-0.763977,1,1,0,0,1,1,...,1,0,0,1,0,0,0,1,0,0
319791,0.238291,-0.424070,-0.490039,-1.460354,0,1,0,0,0,1,...,0,0,1,0,0,0,1,1,0,0
319792,-0.642753,-0.424070,-0.490039,-0.763977,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
319793,0.705560,-0.424070,-0.490039,3.414282,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
X = df_new.drop(['HeartDisease_Yes'], axis=1)
y = df_new[['HeartDisease_Yes']]

In [ ]:
## train,test 분리
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=111)

In [ ]:
print(len(X_train), 'train 수')
print(len(X_test), 'test 수')

223856 train 수
95939 test 수


In [ ]:
#Base Model
#심장병에대한 예측
model=LogisticRegression()
model.fit(X_train, y_train)

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/bda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# 예측
y_pred = model.predict(X_test)
y_pred_prob=model.predict_proba(X_test)[:,1]

##성능평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc =roc_auc_score(y_test, y_pred)

## 혼동행렬, 분류 보고서 출력
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(conf_matrix)
print(class_report)
print(accuracy_score(y_test, y_pred))

[[87019   729]
 [ 7355   836]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     87748
           1       0.53      0.10      0.17      8191

    accuracy                           0.92     95939
   macro avg       0.73      0.55      0.56     95939
weighted avg       0.89      0.92      0.89     95939

0.9157381252670967


- 조금 더 확인해야 하는 부분은 무엇이 있을까?

In [ ]:
# 두 개의 성능 차이를 보고
print('학습셋 모델 정확도', model.score(X_train, y_train))
print('테스트셋 모델 정확도', model.score(X_test, y_test))

학습셋 모델 정확도 0.9159593667357587
테스트셋 모델 정확도 0.9157381252670967


- recall, f1-score
- 0.1 0.17

- accuracy 높은데, recall, precision, f1 은 상대적으로 낮다.

### 클래스이 대한 불균형으로 인해서 성능이 정확도는 높게 나오더라도 다른 지표들이 같이 보고 확인해야 한다.

In [ ]:
y.value_counts()

HeartDisease_Yes
0                   292422
1                    27373
dtype: int64

## 필수과제 1
- 교차검증을 통해서 불균형에 대한 데이터셋을 균형으로 맞추고 학습해서 class 확인 후 모델링

In [ ]:
from imblearn.under_sampling import *

In [ ]:
X_train_re =X_train.copy()
y_train_re =y_train.copy()

In [ ]:
## 임시로 데이터셋 name

X_temp_name = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10',
            'X11','X12','X13','X14','X15','X16','X17','X18','X19','X20',
            'X21','X22','X23','X24','X25','X26','X27','X28','X29','X30',
            'X31','X32','X33','X34','X35','X36','X37']
y_temp_name = ['y1']


X_train_re.columns = X_temp_name
y_train_re.columns = y_temp_name

X_train_re.head()


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37
221542,-0.166045,0.833659,-0.490039,-0.067601,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
144018,-0.391026,-0.424070,-0.490039,-0.067601,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
154685,-0.987305,-0.424070,-0.490039,-0.067601,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
96022,-0.553076,-0.424070,-0.490039,0.628776,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
256866,-0.855148,3.349118,-0.490039,-0.763977,1,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
# 언더샘플링
X_train_under, y_train_under =RandomUnderSampler(random_state=111).fit_resample(X_train_re, y_train_re)

In [ ]:
print(X_train_re.shape, y_train_re.shape)

(223856, 37) (223856, 1)


In [ ]:
print(X_train_under.shape, y_train_under.shape)

(38364, 37) (38364, 1)


In [ ]:
## 언더샘플링 전
y_train_re['y1'].value_counts()

0    204674
1     19182
Name: y1, dtype: int64

In [ ]:
## 언더샘플링 후
## y값의 분포 확인

y_train_under['y1'].value_counts()

0    19182
1    19182
Name: y1, dtype: int64

In [ ]:
## 컬럼명 복구
X_train_under.columns = list(X_train)

In [ ]:
## 컬럼명 복구
y_train_under.columns = list(y_train)

In [ ]:
#Base Model
#심장병에대한 예측
model=LogisticRegression()
model.fit(X_train_under, y_train_under)

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
# 예측
y_pred = model.predict(X_test)
y_pred_prob=model.predict_proba(X_test)[:,1]

##성능평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc =roc_auc_score(y_test, y_pred)

## 혼동행렬, 분류 보고서 출력
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(conf_matrix)
print(class_report)
print(accuracy_score(y_test, y_pred))

[[65791 21957]
 [ 1798  6393]]
              precision    recall  f1-score   support

           0       0.97      0.75      0.85     87748
           1       0.23      0.78      0.35      8191

    accuracy                           0.75     95939
   macro avg       0.60      0.77      0.60     95939
weighted avg       0.91      0.75      0.80     95939

0.7523947508312574


- [87019   729]
-  [ 7355   836]]
>               precision    recall  f1-score   support

               0       0.92      0.99      0.96     87748
               1       0.53      0.10      0.17      8191

        accuracy                           0.92     95939
       macro avg       0.73      0.55      0.56     95939
    weighted avg       0.89      0.92      0.89     95939

- 0.9157381252670967

In [ ]:
# 계수 값도 확인 가능

model.coef_

array([[ 0.05310434,  0.05842839,  0.03792645, -0.0486531 ,  0.40062257,
        -0.17273742,  1.17313848,  0.30436587,  0.76681625, -0.14052093,
         0.19637981,  0.27247658,  0.55821924,  0.96732469,  1.35401136,
         1.57749221,  1.85838576,  2.11117723,  2.46025383,  2.62036867,
         2.9947237 , -0.48368556, -0.2226682 , -0.16899102,  0.00402574,
        -0.10152964,  0.11505285,  0.5751128 ,  0.51934522, -0.02971066,
         1.48480405,  1.01488413,  1.70796971,  0.44860344,  0.35862837,
         0.61367308,  0.13715028]])

In [ ]:
## 로지스틱회귀도 stats 모델에서 Logit 사용해서 summary를 확인할 수 있다.
## 공식적으로 정해진 건 없어서

import statsmodels.api as sm

In [ ]:
ml1=sm.Logit(y_train_under, X_train_under)

In [ ]:
res=ml1.fit(method='newton') # 최적화 방법론

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


/opt/anaconda3/envs/bda/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/envs/bda/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


In [ ]:
res.summary()

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/anaconda3/envs/bda/lib/python3.8/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       HeartDisease_Yes   No. Observations:                38364
Model:                          Logit   Df Residuals:                    38327
Method:                           MLE   Df Model:                           36
Date:                Sun, 21 Jul 2024   Pseudo R-squ.:                     inf
Time:                        18:47:18   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
BMI                                  0.0691      0.013      5.123      0.000       0.043       0.096
PhysicalHealth                       0.0687      0.014      4.862      0.000       0.041       0.096
MentalHealth                         0.0249      0.013      1.864      0.062      -0.001       0.051
SleepTime                           -0.0555      0.012     -4.774      0.000      -0.078      -0.033
Smoking_Yes                          0.3790      0.026     14.636      0.000       0.328       0.430
AlcoholDrinking_Yes                 -0.1987      0.055     -3.590      0.000      -0.307      -0.090
Stroke_Yes                           1.1644      0.054     21.450      0.000       1.058       1.271
DiffWalking_Yes                      0.2594      0.036      7.171      0.000       0.189       0.330
Sex_Male                             0.6930      0.026     26.542      0.000       0.642       0.744
AgeCategory_25-29                   -1.2987      0.127    -10.265      0.000      -1.547      -1.051
AgeCategory_30-34                   -0.9725      0.107     -9.085      0.000      -1.182      -0.763
AgeCategory_35-39                   -0.8925      0.101     -8.857      0.000      -1.090      -0.695
AgeCategory_40-44                   -0.5667      0.093     -6.068      0.000      -0.750      -0.384
AgeCategory_45-49                   -0.1838      0.086     -2.141      0.032      -0.352      -0.016
AgeCategory_50-54                    0.2200      0.079      2.780      0.005       0.065       0.375
AgeCategory_55-59                    0.4576      0.075      6.068      0.000       0.310       0.605
AgeCategory_60-64                    0.7517      0.074     10.225      0.000       0.608       0.896
AgeCategory_65-69                    0.9992      0.073     13.734      0.000       0.857       1.142
AgeCategory_70-74                    1.3542      0.073     18.460      0.000       1.210       1.498
AgeCategory_75-79                    1.5147      0.076     19.852      0.000       1.365       1.664
AgeCategory_80 or older              1.8784      0.076     24.870      0.000       1.730       2.026
Race_Asian                          -2.5840      0.123    -20.935      0.000      -2.826      -2.342
Race_Black                          -2.3095      0.085    -27.205      0.000      -2.476      -2.143
Race_Hispanic                       -2.3008      0.083    -27.852      0.000      -2.463      -2.139
Race_Other                          -2.0855      0.098    -21.218      0.000      -2.278      -1.893
Race_White                          -2.1771      0.072    -30.327      0.000      -2.318      -2.036
Diabetic_No, borderline diabetes     0.1055      0.078      1.352      0.176      -0.047       0.258
Diabetic_Yes                         0.5737      0.034     17.012      0.000       0.508       0.640
Diabetic_Yes (during pregnancy)      0.4541      0.172      2.646      0.008       0.118       0.790
PhysicalA

### 모델 요약 통계 항목 설명

1. **Dep. Variable**: 종속 변수(타겟 변수)의 이름입니다. 여기서는 `HeartDisease_Yes`로, 심장 질환 여부를 나타냅니다.

2. **No. Observations**: 모델에 사용된 관측치(데이터 포인트)의 총 개수입니다. 여기서는 223,856개입니다.

3. **Model**: 사용된 모델의 유형입니다. 여기서는 로지스틱 회귀(`Logit`)입니다.

4. **Df Residuals**: 잔차의 자유도입니다. 이는 총 관측치 수에서 모델에 사용된 파라미터 수를 뺀 값입니다. 여기서는 223,819입니다.

5. **Method**: 모델 추정 방법입니다. 여기서는 최대 우도 추정(MLE, Maximum Likelihood Estimation) 방법을 사용했습니다.

6. **Df Model**: 모델의 자유도입니다. 이는 사용된 독립 변수의 수입니다. 여기서는 36입니다.

7. **Date**: 모델이 적합된 날짜입니다. 여기서는 2024년 1월 21일입니다.

8. **Pseudo R-squ.**: 의사 R-제곱(Pseudo R-squared) 값입니다. 이는 모델의 설명력을 나타내는 지표입니다. 여기서는 0.1811입니다.

9. **Time**: 모델이 적합된 시간입니다. 여기서는 20:50:12입니다.

10. **Log-Likelihood**: 로그 우도(Log-Likelihood) 값입니다. 이는 모델이 데이터를 얼마나 잘 설명하는지 나타냅니다. 값이 클수록 모델이 데이터를 잘 설명합니다. 여기서는 -53,613입니다.

11. **converged**: 모델이 수렴했는지 여부입니다. 여기서는 `True`로, 모델이 성공적으로 수렴했음을 나타냅니다.

12. **LL-Null**: Null 모델의 로그 우도 값입니다. 여기서는 -65,466입니다.

13. **Covariance Type**: 공분산 행렬의 유형입니다. 여기서는 `nonrobust`입니다.

14. **LLR p-value**: 우도비 검정의 p-값입니다. 이는 모델의 유의성을 테스트합니다. p-값이 매우 낮을 경우(일반적으로 0.05보다 작으면), 모델이 통계적으로 유의함을 나타냅니다. 여기서는 p-값이 0.000으로, 모델이 통계적으로 유의합니다.

### 회귀 계수 해석

회귀 계수 표는 각 독립 변수에 대한 정보를 제공합니다. 여기서는 `BMI` 변수를 예로 들어 해석해보겠습니다.

- **coef**: 회귀 계수 값입니다. `BMI` 변수의 회귀 계수는 0.0855입니다. 이는 `BMI`가 1 단위 증가할 때 심장 질환 발생 확률의 로그 오즈가 0.0855만큼 증가함을 의미합니다.
- **std err**: 회귀 계수의 표준 오차입니다. `BMI` 변수의 표준 오차는 0.010입니다.
- **z**: z-값입니다. 이는 회귀 계수를 표준 오차로 나눈 값입니다. `BMI` 변수의 z-값은 8.739입니다.
- **P>|z|**: p-값입니다. 이는 해당 회귀 계수가 0이라는 귀무 가설을 검정합니다. `BMI` 변수의 p-값은 0.000으로, 매우 유의미하다는 것을 나타냅니다.
- **[0.025 0.975]**: 회귀 계수의 95% 신뢰 구간입니다. `BMI` 변수의 신뢰 구간은 [0.066, 0.105]입니다.

### 요약

- `BMI` 변수는 심장 질환 발생 확률과 유의미한 양의 상관관계를 가집니다.
- 모델이 통계적으로 유의하며, `BMI`를 포함한 여러 변수들이 심장 질환 발생에 영향을 미치는 것으로 보입니다.

### 전체 해석

- **모델의 설명력**: Pseudo R-squared 값이 0.1811로, 모델이 데이터의 약 18.11%를 설명합니다. 이는 로지스틱 회귀에서 흔히 낮게 나오는 값이며, 모델이 충분히 유의미한지 다른 지표와 함께 검토해야 합니다.
- **변수의 유의성**: `BMI` 변수는 p-값이 0.000으로 매우 유의미하며, 심장 질환 발생에 긍정적인 영향을 미친다고 해석할 수 있습니다.

## 필수과제 2

- 캐글의 광고데이셋
- clicked_ on Ad y값
- 나머지 변수들 사이트 체류시간, 인컴, 광고, 지역이나 기타 등등
- 0,1 분류하는 코드
- 데이터 시각화 하시고, 피처 정리하고, 왜 이 피처의 이런 전처리 작업을 했는지 정리하여서
- 실제 성능을 모델링하여 공유해 주시면 됩니다.
- 코드와 함께 주석으로 같이 과제 제출해 주시고, 교차검증 같이 진행하기, 클래스 불균형까지도 확인해서 성능 비교하는 것으로!

- 전처리 후
---
- 하이퍼파라미터를 조금 더 개선해서 모델링의 성능이 올라가는지?
- 이 부분까지도 같이 포함해 주세요.

- **꼭 포함해야 하는 항목 : 데이터 전처리 시각화, 피처 엔지니어링, 교차검증( 클래스 불균형까지 확인해서 ), 하이퍼파라미터 튜닝을 통한 성능 개선 여부**